Next steps:

 - ~load ROIs~ (or create one with shape of the image)
 - ~make adaptations to database:~
   - ~use Path objects to enable continous integration~
   - ~rename deepflash2_dir to segmentation_tool_dir~
   - ~create sorted list of all preprocessing steps (i.e. preprocessing strategies)~
 - ~adapt main~
 - ~function that saves preprocessed images~
 
 
 - confirm that roi is loaded and adjusted correctly (rows and columns switched?)
 - continue with segmentations

 - Minimal preprocessing steps are:
    - save the "unprocessed" microscopy images to the preprocessed_dir
    - load the unprocessed ROIs into the database (create ROI with shape of image if whole image is to be analyzed)

In [6]:
import numpy as np
max(np.asarray([0, 1, 3, False, False]))

3

In [ ]:
from findmycells.main import Project
from pathlib import Path

In [ ]:
from findmycells.preprocessing import CropStitchingArtefactsRGB, ConvertFrom12To8BitRGB

In [ ]:
project_root_dir = Path('/mnt/c/Users/dsege/TEMP/test_project4')

In [ ]:
user_input_via_gui = {'project_root_dir': project_root_dir, 
                      'preprocessing_strategies': [ConvertFrom12To8BitRGB, CropStitchingArtefactsRGB]}

In [ ]:
refactored_project = Project({'project_root_dir': project_root_dir})

In [ ]:
refactored_project.load_status()

In [ ]:
refactored_project.database.file_infos

In [ ]:
refactored_project.preprocess()

In [ ]:
refactored_project.save_status()

In [ ]:
refactored_project.database.area_rois_for_quantification

In [ ]:
refactored_project.preprocessing_object.load_rois()

In [ ]:
refactored_project.database.file_infos['microscopy_filepath'][0]

In [ ]:
from findmycells.microscopyimages import MicroscopyImageLoader
import matplotlib.pyplot as plt

In [ ]:
microscopy_image_loader = MicroscopyImageLoader(filepath = refactored_project.database.file_infos['microscopy_filepath'][0],
                                                filetype = refactored_project.database.file_infos['microscopy_filetype'][0])
zstack = microscopy_image_loader.as_array()

In [ ]:
zstack.shape

In [ ]:
cropped_zstack = zstack[:, 100:7100, 100:4600, :]

In [ ]:
from PIL import Image
import os

In [ ]:
pil_image = Image.fromarray(rgb_image, 'RGB')

In [ ]:
pil_image.save('pil_image.png')

In [ ]:
from skimage.io import imsave
rgb_image = rgb_image.astype('uint8')
imsave('skimage_image.png', rgb_image)

In [ ]:
rgb_image

In [ ]:
rgb_image = refactored_project.preprocessing_object.preprocessed_image[0].copy()

In [ ]:
plt.imshow(rgb_image)

In [ ]:
converted_image = (rgb_image / 4095 * 255).round(0).astype('uint8')

In [ ]:
from pathlib import Path
from shapely.geometry import Polygon
import roifile
import numpy as np

In [ ]:
roi_filepath = Path('/mnt/c/Users/dsege/TEMP/test_project2/01_rois_to_analyze/tg/#5_tg_old/#hund.roi')

In [ ]:
roi_filepath = Path('/mnt/c/Users/dsege/TEMP/test_project2/01_rois_to_analyze/tg/#5_tg_old/RoiSet.zip')

In [ ]:
roi_filepath.name.endswith('.zip')

In [ ]:
'.zip' in ['.roi', '.zip']

In [ ]:
loaded_rois[1].name

In [ ]:
if roi_filepath.name.endswith('.roi'):
    loaded_rois = [roifile.ImagejRoi.fromfile(roi_filepath)]
elif roi_filepath.name.endswith('.zip'):
    loaded_rois = roifile.ImagejRoi.fromfile(roi_filepath)
else:
    filetype = roi_filepath.name[roi_filepath.name.find('.'):]
    raise ValueError(f'ImageJROIReader cannot handle files of type {filetype}')

rois_as_shapely_polygons = {'all_planes': dict()}
roi_count = len(loaded_rois)
for roi_index in range(roi_count):
    row_coords = loaded_rois[roi_index].coordinates()[:, 1]
    col_coords = loaded_rois[roi_index].coordinates()[:, 0]
    if roi_count > 1:
        rois_as_shapely_polygons['all_planes'][loaded_rois[roi_index].name] = Polygon(np.asarray(list(zip(row_coords, col_coords))))
    else:
        rois_as_shapely_polygons['all_planes'][str(roi_index).zfill(3)] = Polygon(np.asarray(list(zip(row_coords, col_coords)))) 


In [ ]:
row_coords

In [ ]:
row_coords = [row - 5 for row in row_coords]
row_coords

In [ ]:
a = np.asarray(row_coords)

In [ ]:
a -= 5

In [ ]:
a

In [ ]:
Polygon(np.asarray(list(zip(np.asarray(row_coords), np.asarray(col_coords)))))

In [ ]:
col_coords

In [ ]:
row_coords = [coordinates[0] for coordinates in rois_as_shapely_polygons['all_planes']['0330-0403'].boundary.coords[:]]
col_coords = [coordinates[1] for coordinates in rois_as_shapely_polygons['all_planes']['0330-0403'].boundary.coords[:]]

In [ ]:
rois_as_shapely_polygons['all_planes'].keys()

In [ ]:
f'ImageJROIReader cannot handle files of type {filetype}'

In [ ]:
roi[0].coordinates()

In [ ]:
self.as_polygons[roi_id] = Polygon(np.asarray(list(zip(self.roi_coordinates[roi_id][0], self.roi_coordinates[roi_id][1]))))

In [ ]:
rois_as_shapely_polygons['file_id']['plane_id']['roi_id']

In [ ]:
hunde = list()
katzen = list()
for x in hunde:
    katzen.append(x)
    

In [ ]:
katzen

In [ ]:
project_root_dir = Path('/mnt/c/Users/dsege/TEMP/test_project2/')

In [ ]:
project_root_dir.joinpath(elem).as_posix()

In [ ]:
elem

In [ ]:
project_root_dir.is_dir()

In [ ]:
project_root_dir

In [ ]:
self.project_root_dir + [elem for elem in subdirectories if 'microscopy' in elem][0] + '/'

In [ ]:
subdirectories

In [ ]:
import os
def listdir_nohidden(path):
    return [f for f in os.listdir(path) if f.startswith('.') == False]

In [ ]:
subdirectories = listdir_nohidden(path = project_root_dir)

In [ ]:
? microscopy_image_dir.mkdir

In [ ]:
subdirectory_attributes = {'rois_to_analyze_dir': {'foldername': '01_rois_to_analyze',
                                                   'key_substring': 'rois'},
                           'preprocessed_images_dir': {'foldername': '02_preprocessed_images',
                                                       'key_substring': 'preprocessed'},
                           'deepflash2_dir': {'foldername': '03_deepflash2',
                                              'key_substring': 'deepflash'},
                           'binary_segmentations_dir': {'foldername': '04_binary_segmentations',
                                                        'key_substring': 'binary'},
                           'instance_segmentations_dir': {'foldername': '05_instance_segmentations',
                                                          'key_substring': 'instance'},
                           'inspection_dir': {'foldername': '06_inspection',
                                              'key_substring': 'inspection'},
                           'results_dir': {'foldername': '07_results',
                                           'key_substring': 'results'}}


In [ ]:
for attribute_key in subdirectory_attributes.keys():
    elements_matching_key_substring = [elem for elem in subdirectories if subdirectory_attributes[attribute_key]['key_substring'] in elem]
    if len(elements_matching_key_substring) > 0:
        for matching_element in elements_matching_key_substring:
            if (project_root_dir.joinpath(matching_element).is_dir()) & (hasattr(self, attribute_key) == False):
                setattr(self, attribute_key, project_root_dir.joinpath(matching_element))
    if hasattr(self, attribute_key) == False:
        subdirectory_path = project_root_dir.joinpath(subdirectory_attributes[attribute_key]['foldername'])
        subdirectory_path.mkdir()
        setattr(self, attribute_key, subdirectory_path)
                
                

In [ ]:
folder = [elem for elem in subdirectories if 'microscopy' in elem][0]

In [ ]:
microscopy_image_dir = project_root_dir.joinpath('hund')

In [ ]:
project_root_dir.joinpath('hund').is_dir()
    

In [ ]:
microscopy_image_dir

In [ ]:
from findmycells.main import Project
import os
import pandas as pd
from datetime import datetime